In [83]:
import pandas as pd
# no cut off the output
pd.set_option('display.max_rows', None)

chars = pd.read_csv('characters_freq.csv')
chars.head()

,index,level,character,pinyin
0,7.79,一级,的,de
1,7.79,四级,的,dí
2,7.79,二级,的,dì
3,7.17,一级,是,shì
4,7.16,一级,在,zài


In [84]:
# group by level column

chars.groupby('level').count()

,index,character,pinyin
level,,,
一级,307,307,307
三级,310,310,310
二级,321,321,321
五级,316,316,316
六级,316,316,316
四级,323,323,323
高等,1256,1256,1256


In [94]:
import pandas as pd
import unicodedata
# open harry.txt
with open('text_to_analyse.txt', 'r') as f:
    text = f.read()

# count characters
from collections import Counter
counts = Counter(text)

# convert to dataframe
counts = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts.columns = ['character', 'count']

counts = counts.sort_values(by='count', ascending=False)

counts.head()

,character,count
95,,153778
11,\n,77194
32,的,74910
76,。,56182
126,他,42563


In [100]:
import unicodedata


def is_han(char):
    try:
        return all('CJK UNIFIED' in unicodedata.name(c) for c in char)
    except:
        return False


# filter out non-Chinese characters
counts['is_chinese'] = counts['character'].apply(is_han)
counts = counts[counts['is_chinese'] == True]

In [101]:

def filter_by_level(level):
    return counts[counts['character'].isin(chars[chars['level'] == level]['character'])].sort_values('count', ascending=False)


# '一级', '二级', '三级', '四级', '五级', '六级',
for level in ['高等']:
    print(level)
    print(filter_by_level(level))
    print()

高等
     character  count  is_chinese
308          着  16641        True
258          和   7761        True
108          斯   7730        True
9            罗   6545        True
1600         魔   5398        True
1980         赫   3416        True
3121         邓   2411        True
1300         杖   1912        True
761          伯   1503        True
234          别   1493        True
1561         霍   1466        True
1700         荷   1466        True
1021         巫   1445        True
726          将   1237        True
658          丹   1081        True
2433         伏   1048        True
810          迪   1042        True
1302         尼   1028        True
1351         瑞    971        True
1104         狼    933        True
1275         兰    846        True
649          姆    840        True
360          片    822        True
47           提    814        True
1997         乔    802        True
2101         芬    794        True
703          哦    737        True
36           传    693        True
734        

In [102]:
# print the chars that are not in the list of chars

non_in_hsk = counts[~counts['character'].isin(
    chars['character'])].sort_values('count', ascending=False)
non_in_hsk.head()

,character,count,is_chinese
958,莱,1637,True
847,弗,1620,True
682,妮,1260,True
1645,咒,1180,True
1651,韦,890,True


In [106]:
# print a table of percentage of each level of the full text

# get the total count
total_count = counts['count'].sum()

# get the percentage
counts = counts.copy()  # create a copy of the dataframe to avoid SettingWithCopyWarning
counts['percentage'] = counts['count'] / total_count * 100

# get the level of each character
counts['level'] = counts['character'].apply(get_level)


def get_level(character):
    try:
        return chars[chars['character'] == character]['level'].values[0]
    except:
        return 'N/A'


counts['level'] = counts['character'].apply(get_level)

# group by level
counts.groupby('level').sum()

# print the percentage of each level
counts.groupby('level').sum()['percentage'].sort_values(ascending=False)

level
一级     50.670184
二级     17.260424
三级     11.426207
高等      6.683129
四级      5.978635
五级      3.309374
六级      3.240342
N/A     1.431704
Name: percentage, dtype: float64

In [104]:
counts[~counts['character'].isin(chars['character'])].sort_values(
    'count', ascending=False).head()

,character,count,is_chinese,percentage,level
958,莱,1637,True,0.096338,N/A
847,弗,1620,True,0.095337,N/A
682,妮,1260,True,0.074151,N/A
1645,咒,1180,True,0.069443,N/A
1651,韦,890,True,0.052377,N/A
